# Mehanizmi pozornosti in transformatorji

Ena glavnih pomanjkljivosti rekurentnih mrež je, da imajo vse besede v zaporedju enak vpliv na rezultat. To povzroča suboptimalno delovanje pri standardnih LSTM modelih kodirnik-dekodirnik za naloge, kot sta prepoznavanje imenovanih entitet in strojno prevajanje. V resnici imajo določene besede v vhodnem zaporedju pogosto večji vpliv na izhodne rezultate kot druge.

Razmislimo o modelu zaporedje-v-zaporedje, kot je strojno prevajanje. Ta model je implementiran z dvema rekurentnima mrežama, kjer ena mreža (**kodirnik**) stisne vhodno zaporedje v skrito stanje, druga mreža (**dekodirnik**) pa to skrito stanje razširi v preveden rezultat. Težava pri tem pristopu je, da ima končno stanje mreže težave z zapomnitvijo začetka stavka, kar povzroča slabo kakovost modela pri dolgih stavkih.

**Mehanizmi pozornosti** omogočajo tehtanje kontekstualnega vpliva vsakega vhodnega vektorja na vsako izhodno napoved RNN. To se implementira z ustvarjanjem bližnjic med vmesnimi stanji vhodne RNN in izhodne RNN. Na ta način pri generiranju izhodnega simbola $y_t$ upoštevamo vsa vhodna skrita stanja $h_i$ z različnimi utežnimi koeficienti $\alpha_{t,i}$.

![Slika, ki prikazuje model kodirnik/dekodirnik z aditivno plastjo pozornosti](../../../../../translated_images/sl/encoder-decoder-attention.7a726296894fb567.webp)
*Model kodirnik-dekodirnik z mehanizmom aditivne pozornosti v [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), povzeto iz [tega bloga](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Matrika pozornosti $\{\alpha_{i,j}\}$ predstavlja stopnjo, do katere določene vhodne besede vplivajo na generacijo določene besede v izhodnem zaporedju. Spodaj je primer takšne matrike:

![Slika, ki prikazuje vzorčno poravnavo, najdeno z RNNsearch-50, povzeto iz Bahdanau - arviz.org](../../../../../translated_images/sl/bahdanau-fig3.09ba2d37f202a6af.webp)

*Slika povzeta iz [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Slika 3)*

Mehanizmi pozornosti so odgovorni za velik del trenutnega ali skoraj trenutnega stanja umetnosti na področju obdelave naravnega jezika. Dodajanje pozornosti pa močno poveča število parametrov modela, kar je povzročilo težave pri skaliranju RNN. Ključna omejitev pri skaliranju RNN je, da rekurentna narava modelov otežuje združevanje in paralelizacijo učenja. V RNN je treba vsak element zaporedja obdelati v zaporednem vrstnem redu, kar pomeni, da ga ni mogoče enostavno paralelizirati.

Sprejetje mehanizmov pozornosti v kombinaciji s to omejitvijo je privedlo do nastanka zdajšnjih transformatorjev, ki predstavljajo stanje umetnosti, kot so BERT, OpenGPT3 in drugi.

## Transformatorji

Namesto da bi kontekst vsake prejšnje napovedi posredovali v naslednji korak ocenjevanja, **transformatorji** uporabljajo **pozicijske kodiranja** in pozornost za zajemanje konteksta določenega vhoda znotraj določenega okna besedila. Spodnja slika prikazuje, kako pozicijska kodiranja s pozornostjo zajamejo kontekst znotraj določenega okna.

![Animiran GIF, ki prikazuje, kako se izvajajo ocene v transformatorjih.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Ker je vsak vhodni položaj neodvisno preslikan na vsak izhodni položaj, lahko transformatorji bolje paralelizirajo kot RNN, kar omogoča veliko večje in izrazitejše jezikovne modele. Vsaka glava pozornosti se lahko uporablja za učenje različnih odnosov med besedami, kar izboljša naloge obdelave naravnega jezika.

**BERT** (Bidirectional Encoder Representations from Transformers) je zelo velik večplastni transformator z 12 plastmi za *BERT-base* in 24 za *BERT-large*. Model je najprej predhodno usposobljen na velikem korpusu besedilnih podatkov (Wikipedia + knjige) z uporabo nenadzorovanega učenja (napovedovanje zamaskiranih besed v stavku). Med predhodnim učenjem model pridobi pomembno raven razumevanja jezika, ki jo je nato mogoče uporabiti z drugimi nabori podatkov z uporabo prilagoditve. Ta proces se imenuje **prenosno učenje**.

![Slika s http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/sl/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Obstaja veliko različic arhitektur transformatorjev, vključno z BERT, DistilBERT, BigBird, OpenGPT3 in drugimi, ki jih je mogoče prilagoditi. Paket [HuggingFace](https://github.com/huggingface/) ponuja repozitorij za učenje mnogih teh arhitektur s PyTorch.

## Uporaba BERT za klasifikacijo besedila

Poglejmo, kako lahko uporabimo predhodno usposobljen model BERT za reševanje naše tradicionalne naloge: klasifikacije zaporedij. Klasificirali bomo naš izvirni nabor podatkov AG News.

Najprej naložimo knjižnico HuggingFace in naš nabor podatkov:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Ker bomo uporabljali vnaprej naučen BERT model, bomo morali uporabiti določen tokenizer. Najprej bomo naložili tokenizer, povezan z vnaprej naučenim BERT modelom.

Knjižnica HuggingFace vsebuje repozitorij vnaprej naučenih modelov, ki jih lahko uporabite tako, da njihove imena podate kot argumente funkcijam `from_pretrained`. Vsi potrebni binarni datoteki za model bodo samodejno preneseni.

V določenih primerih pa boste morali naložiti svoje modele. V tem primeru lahko določite imenik, ki vsebuje vse ustrezne datoteke, vključno s parametri za tokenizer, datoteko `config.json` s parametri modela, binarnimi utežmi itd.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` objekt vsebuje funkcijo `encode`, ki jo je mogoče neposredno uporabiti za kodiranje besedila:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Potem ustvarimo iteratorje, ki jih bomo uporabljali med treningom za dostop do podatkov. Ker BERT uporablja svojo lastno funkcijo kodiranja, bi morali definirati funkcijo za dodajanje presledkov, podobno kot `padify`, ki smo jo definirali prej:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

V našem primeru bomo uporabili vnaprej naučen BERT model, imenovan `bert-base-uncased`. Naložimo model z uporabo paketa `BertForSequenceClassification`. To zagotavlja, da ima naš model že zahtevano arhitekturo za klasifikacijo, vključno s končnim klasifikatorjem. Prikazalo se bo opozorilno sporočilo, ki navaja, da uteži končnega klasifikatorja niso inicializirane in da model potrebuje predhodno učenje - to je povsem v redu, saj je točno to tisto, kar bomo storili!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Zdaj smo pripravljeni na začetek učenja! Ker je BERT že predhodno naučen, želimo začeti z razmeroma majhno hitrostjo učenja, da ne bi uničili začetnih uteži.

Vso zahtevno delo opravi model `BertForSequenceClassification`. Ko model pokličemo na učnih podatkih, vrne tako izgubo kot izhod omrežja za vhodni minibatch. Izgubo uporabimo za optimizacijo parametrov (`loss.backward()` izvede povratno propagacijo), medtem ko `out` uporabimo za izračun učne natančnosti, tako da primerjamo dobljene oznake `labs` (izračunane z uporabo `argmax`) s pričakovanimi `labels`.

Za nadzor procesa akumuliramo izgubo in natančnost skozi več iteracij ter ju izpišemo vsakih `report_freq` učnih ciklov.

To učenje bo verjetno trajalo kar nekaj časa, zato omejimo število iteracij.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Vidite lahko (še posebej, če povečate število iteracij in počakate dovolj dolgo), da nam klasifikacija z BERT daje precej dobro natančnost! To je zato, ker BERT že zelo dobro razume strukturo jezika, mi pa moramo le prilagoditi končni klasifikator. Vendar pa, ker je BERT velik model, celoten proces učenja traja dolgo in zahteva veliko računske moči! (GPU, in po možnosti več kot enega).

> **Opomba:** V našem primeru uporabljamo enega najmanjših vnaprej naučenih BERT modelov. Obstajajo večji modeli, ki bodo verjetno dali boljše rezultate.


## Ocena zmogljivosti modela

Zdaj lahko ocenimo zmogljivost našega modela na testnem naboru podatkov. Zanka za ocenjevanje je precej podobna zanki za učenje, vendar ne smemo pozabiti preklopiti modela v način ocenjevanja z uporabo `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Ključne točke

V tej enoti smo videli, kako enostavno je vzeti vnaprej naučen jezikovni model iz knjižnice **transformers** in ga prilagoditi naši nalogi razvrščanja besedil. Podobno lahko modele BERT uporabimo za ekstrakcijo entitet, odgovarjanje na vprašanja in druge naloge obdelave naravnega jezika.

Transformers modeli predstavljajo trenutno najsodobnejši pristop v obdelavi naravnega jezika (NLP), in v večini primerov bi morali biti prva rešitev, s katero začnete eksperimentirati pri implementaciji prilagojenih NLP rešitev. Vendar pa je razumevanje osnovnih principov rekurentnih nevronskih mrež, o katerih smo govorili v tem modulu, izjemno pomembno, če želite graditi napredne nevronske modele.



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da se zavedate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo strokovno človeško prevajanje. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki izhajajo iz uporabe tega prevoda.
